In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [2]:
text = open('evgenyi_onegin.txt', 'rb').read().decode(encoding='utf-8')

print(f'Length of text: {len(text)} characters')

Length of text: 286984 characters


In [3]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
     

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [9]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [12]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 131)


In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(64, None)]         0           []                               
                                                                                                  
 embedding (Embedding)          (64, None, 256)      33536       ['input_1[0][0]']                
                                                                                                  
 gru (GRU)                      (64, None, 1024)     3938304     ['embedding[0][0]']              
                                                                                                  
 concatenate (Concatenate)      (64, None, 1280)     0           ['embedding[0][0]',              
                                                                  'gru[0][0]']                

In [15]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 0.04687095,  0.02929705, -0.03138408, ..., -0.02342152,
         0.02640591,  0.00098065],
       [ 0.0565406 ,  0.02509448, -0.04665244, ..., -0.02624841,
         0.02574374, -0.00127318],
       [ 0.06203498,  0.01919578, -0.05622396, ..., -0.02936875,
         0.02446199, -0.0036685 ],
       ...,
       [ 0.07882039,  0.01429872, -0.03623161, ..., -0.02716543,
         0.00709531, -0.00152817],
       [ 0.07815094,  0.01417261, -0.050198  , ..., -0.02893178,
         0.01443203, -0.00331788],
       [ 0.07581208,  0.01140672, -0.05847971, ..., -0.03116653,
         0.01794451, -0.00496988]], dtype=float32)>

In [16]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [17]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("-------------------------------------")
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '                 И вот с холма Татьяна сходит,\n                        Едва дыша; кругом обводит\n   '
-------------------------------------
Next Char Predictions: 
 'qpдnъY?йрddС(:w,О4йТH3mzPC5ппйD}Чат4"ХlыСи-SщЕRRzLЕ\'mnтЕИgЖdv4нs5uюыGх2уФOm1шаяd8wОиСаЯйм3 2кэ{NdГбс'


In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())
     

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8738785


In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [22]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
44/44 [==============================] - 342s 8s/step - loss: 2.2448
Epoch 2/10
44/44 [==============================] - 335s 8s/step - loss: 1.5442
Epoch 3/10
44/44 [==============================] - 340s 8s/step - loss: 1.3550
Epoch 4/10
44/44 [==============================] - 341s 8s/step - loss: 1.2860
Epoch 5/10
44/44 [==============================] - 337s 8s/step - loss: 1.2480
Epoch 6/10
44/44 [==============================] - 337s 8s/step - loss: 1.2043
Epoch 7/10
44/44 [==============================] - 337s 8s/step - loss: 1.1753
Epoch 8/10
44/44 [==============================] - 6054s 141s/step - loss: 1.1400
Epoch 9/10
44/44 [==============================] - 339s 8s/step - loss: 1.1008
Epoch 10/10
44/44 [==============================] - 338s 8s/step - loss: 1.0603


In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 131)


In [24]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [25]:
text_ = generate_text(model, start_string=u"Хранили многие ")
print(text_)

Хранили многие подальной
                                                                                                                                                                                                                                                                                 Не долого постралая пестов,
                                                                                                                                                                                             


In [26]:
text_ = generate_text(model, start_string=u"Я плачу ")
print(text_)

Я плачу содет.

                                                                                                   XVI

                                                                 Стат кольчи с не омони свута.

                                                                                                                   XXII

                                                                                                                                                                           
